In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import Dataset,load_dataset
import ast
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support
from torch.utils.data import DataLoader
from transformers import default_data_collator,TrainerCallback,set_seed
import torch
seed = 42
set_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlbse-datasets/NLBSE_Dataset_Java.csv
/kaggle/input/nlbse-datasets/NLBSE_Dataset_Python.csv
/kaggle/input/nlbse-datasets/NLBSE_Dataset_Pharo.csv
Using device: cuda


In [2]:
df = pd.read_csv('/kaggle/input/nlbse-datasets/NLBSE_Dataset_Java.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47169 entries, 0 to 47168
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             47169 non-null  object
 1   comment_sentence  47168 non-null  object
 2   labels            47169 non-null  object
 3   types             47169 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [4]:
df.head(10)
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Wandb")
import wandb

# Replace YOUR_API_KEY with your actual API key
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
df.drop_duplicates(subset=['comment_sentence'], keep='first', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36200 entries, 0 to 47165
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             36200 non-null  object
 1   comment_sentence  36199 non-null  object
 2   labels            36200 non-null  object
 3   types             36200 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [6]:
null_rows = df[df['comment_sentence'].isnull()]

print("Rows with null values in 'comment_sentence':")
print(null_rows)

Rows with null values in 'comment_sentence':
                                   class comment_sentence           labels  \
30289  GDBDynamicPrintfPropertyPage.java              NaN  [1 0 0 0 0 0 0]   

      types  
30289    hi  


In [7]:
df_cleaned = df.dropna(subset=['comment_sentence'])
print("DataFrame shape after removing nulls:", df_cleaned.shape)
df_cleaned.info()

DataFrame shape after removing nulls: (36199, 4)
<class 'pandas.core.frame.DataFrame'>
Index: 36199 entries, 0 to 47165
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   class             36199 non-null  object
 1   comment_sentence  36199 non-null  object
 2   labels            36199 non-null  object
 3   types             36199 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [8]:
pattern = r'https?://\S+|\t'
rows_with_pattern = df_cleaned.apply(lambda row: row.astype(str).str.contains(pattern).any(), axis=1)

# Count rows with patterns
num_rows_with_pattern = rows_with_pattern.sum()
print(f"\nNumber of rows containing patterns: {num_rows_with_pattern}")

# Remove `//` or `*` from all columns
df_cleaned = df_cleaned.replace(pattern, '', regex=True)


Number of rows containing patterns: 1474


In [9]:
df = df_cleaned
df['combo'] = df['comment_sentence'] +"  |  "+  df['class']
java_dataset = Dataset.from_pandas(df)
# Split the dataset into train and validation subsets
train_test_split = java_dataset.train_test_split(test_size=0.2, seed=42)

# Extract train and validation datasets
java_train = train_test_split['train']
java_test = train_test_split['test']
java_labels = ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational']

In [10]:
# Use Hugging Face's default data collator
data_collator = default_data_collator
num_labels = len(java_labels)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["combo"], truncation=True, padding="max_length", max_length=128)
tokenized_train = java_train.map(tokenize_function, batched=True)
tokenized_test = java_test.map(tokenize_function, batched=True)

# Convert labels to tensors
def encode_labels(examples):
    if isinstance(examples['labels'], str):
        examples["labels"]=examples["labels"].replace(" ", ",")
        labels = ast.literal_eval(examples['labels'])
    else:
        labels = examples['labels']
    # Convert labels to tensors
    labels = torch.tensor(labels, dtype=torch.float32)
    return {'labels': labels}
tokenized_train = tokenized_train.map(encode_labels)
tokenized_test = tokenized_test.map(encode_labels)

# Format datasets for PyTorch
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-large",
    num_labels=num_labels,
    problem_type="multi_label_classification"
).to(device)

# Define evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = (logits > 0.5).astype(int)  # Multi-label threshold
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    return {"precision": precision, "recall": recall, "f1": f1}


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results1",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2.01916468009019e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=16,
    weight_decay= 0.09130339417806095,
    logging_dir="./logs1",
    logging_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Use validation loss to select the best model
    greater_is_better=False,  # Lower validation loss is better
)


# Custom callback to monitor F1 score and restart training if F1 is zero
class RestartIfF1ZeroCallback(TrainerCallback):
    def __init__(self, trainer, train_dataset, eval_dataset, tokenizer, data_collator):
        self.trainer = trainer
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset
        self.tokenizer = tokenizer
        self.data_collator = data_collator

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        # Check F1 score
        if "eval_f1" in metrics and metrics["eval_f1"] == 0.0:
            print("F1 score is zero. Restarting training...")
            
            # Reinitialize model
            self.trainer.model = AutoModelForSequenceClassification.from_pretrained(
                "roberta-base",
                num_labels=num_labels,
                problem_type="multi_label_classification"
            ).to(device)

            # Restart training
            self.trainer.train_dataset = self.train_dataset
            self.trainer.eval_dataset = self.eval_dataset
            self.trainer.tokenizer = self.tokenizer
            self.trainer.data_collator = self.data_collator

            self.trainer.train()

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Add custom callback
trainer.add_callback(RestartIfF1ZeroCallback(trainer, tokenized_train, tokenized_test, tokenizer, data_collator))

# Train the model
trainer.train()

# Save the model
trainer.save_model("./roberta-java-multi-label")
tokenizer.save_pretrained("./roberta-java-multi-label")

# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_23/1696731548.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wan

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.098900,0.089052,0.928023,0.864194,0.894972
2,0.052700,0.051253,0.960762,0.911396,0.935428
3,0.030600,0.039749,0.969347,0.951045,0.960109
4,0.021500,0.036699,0.970448,0.961025,0.965714
5,0.012800,0.032632,0.974952,0.965880,0.970395
6,0.008200,0.031193,0.980269,0.971544,0.975887
7,0.007000,0.026548,0.982777,0.977343,0.980053
8,0.002600,0.031657,0.978440,0.979231,0.978835
9,0.003800,0.026650,0.983613,0.979501,0.981553
10,0.002700,0.025163,0.984050,0.981794,0.982920


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

Evaluation Results: {'eval_loss': 0.024021035060286522, 'eval_precision': 0.9857896873731222, 'eval_recall': 0.9823331085637221, 'eval_f1': 0.9840583626047014, 'eval_runtime': 99.5871, 'eval_samples_per_second': 72.7, 'eval_steps_per_second': 1.145, 'epoch': 16.0}


In [11]:
import numpy as np
import pandas as pd
import time
import torch

def evaluate_roberta(new_dataset, model, tokenizer, labels, batch_size=16, device='cuda'):
    """
    Evaluate a fine-tuned RoBERTa-large model on a new dataset.

    Args:
        new_dataset (dict): A dictionary with 'combo' (input texts) and 'labels' (one-hot encoded labels).
        model (transformers.PreTrainedModel): A trained RoBERTa model.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer corresponding to the model.
        labels (list): List of label categories.
        batch_size (int): Batch size for evaluation.
        device (str): Device to use ('cuda' or 'cpu').

    Returns:
        pd.DataFrame: DataFrame containing precision, recall, and F1-score for each label.
        float: Average runtime per batch.
    """
    # Move model to device
    model.to(device)

    # Prepare data
    texts = new_dataset['combo']
    true_labels = np.array(new_dataset['labels'])

    # Tokenize the inputs
    inputs = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Split into batches
    num_batches = (len(texts) + batch_size - 1) // batch_size
    predictions = []
    start_time = time.time()

    # Perform inference in batches
    for i in range(num_batches):
        batch_input_ids = input_ids[i * batch_size: (i + 1) * batch_size]
        batch_attention_mask = attention_mask[i * batch_size: (i + 1) * batch_size]

        with torch.no_grad():
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            preds = (logits.sigmoid() > 0.5).int().cpu().numpy()
            predictions.append(preds)
            
    end_time = time.time()
    avg_runtime = (end_time - start_time) / num_batches

    # Concatenate predictions
    predictions = np.vstack(predictions)

    # Evaluate metrics for each label
    metrics = []
    for i, label in enumerate(labels):
        tp = np.sum((true_labels[:, i] == 1) & (predictions[:, i] == 1))
        fp = np.sum((true_labels[:, i] == 0) & (predictions[:, i] == 1))
        fn = np.sum((true_labels[:, i] == 1) & (predictions[:, i] == 0))
        tn = np.sum((true_labels[:, i] == 0) & (predictions[:, i] == 0))

        precision = tp / (tp + fp) if tp + fp > 0 else 0
        recall = tp / (tp + fn) if tp + fn > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics.append({'label': label, 'precision': precision, 'recall': recall, 'f1': f1})

    # Convert metrics to DataFrame
    metrics_df = pd.DataFrame(metrics)
    return metrics_df, avg_runtime

In [12]:
import re
# Use the model and tokenizer from the training session
trained_model = model  # Replace with your trained model instance
trained_tokenizer = tokenizer  # Replace with your tokenizer instance
test = load_dataset('NLBSE/nlbse25-code-comment-classification')['java_test']
# def clean_text(example):
#     if "combo" in example:  # Replace "text" with the relevant column name
#         example["combo"] = re.sub(r'https?://\S+|\t', '', example["combo"], flags=re.MULTILINE)
#         example["combo"] = example["combo"].strip()  # Remove leading/trailing spaces
#     return example
# test = test.map(clean_text)
# Define label names
labels =  ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational']

# Evaluate the model
metrics, avg_runtime = evaluate_roberta(test, trained_model, trained_tokenizer, labels)

print("Evaluation Metrics:")
print(metrics)
print(f"Average runtime per batch: {avg_runtime:.4f} seconds")
from huggingface_hub import login

# Retrieve API token from Kaggle secrets
import os
huggingface_token = user_secrets.get_secret("Huggingface")

login(token=huggingface_token)
trained_model.push_to_hub("NLBSE-Java-final")
trained_tokenizer.push_to_hub("NLBSE-Java-final")

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

java_train-00000-of-00001.parquet:   0%|          | 0.00/680k [00:00<?, ?B/s]

java_test-00000-of-00001.parquet:   0%|          | 0.00/174k [00:00<?, ?B/s]

python_train-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

python_test-00000-of-00001.parquet:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

...

Generating pharo_test split:   0%|          | 0/289 [00:00<?, ? examples/s]

Evaluation Metrics:
         label  precision    recall        f1
0      summary   0.904488  0.881166  0.892675
1    Ownership   1.000000  1.000000  1.000000
2       Expand   0.439252  0.460784  0.449761
3        usage   0.921951  0.877030  0.898930
4      Pointer   0.806452  0.951087  0.872818
5  deprecation   0.818182  0.600000  0.692308
6     rational   0.268293  0.323529  0.293333
Average runtime per batch: 1.5075 seconds


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MushfiqurRR/NLBSE-Java-final/commit/a05f32f43abf344cabe75454e45560606fce6156', commit_message='Upload tokenizer', commit_description='', oid='a05f32f43abf344cabe75454e45560606fce6156', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MushfiqurRR/NLBSE-Java-final', endpoint='https://huggingface.co', repo_type='model', repo_id='MushfiqurRR/NLBSE-Java-final'), pr_revision=None, pr_num=None)